In [1]:
# Import statements
import serpapi
import requests
from transformers import BertTokenizer, BertForSequenceClassification
import torch

c:\Users\bryan\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\bryan\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [23]:
# Set device to whatever device is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using GPU')
else:
    device = torch.device('cpu')
    print('Using CPU')

Using CPU


In [24]:
# Step 1: Search Google News
def search_google_news(query, num_results=10):
    print(f"Searching Google News for: {query}")
    output = []
    params = {
        "engine": "google_news",
        "q": query,
        "api_key": "7d2578c0d84947939e1b3aab051d37a70c90e91a9ae0f3c2f98001559aba08f5"
    }
    print("Calling SerpAPI")
    result = serpapi.search(params)
    print("Going through articles")
    for article in result["news_results"]:
      output.append(article["title"])
    return result

In [25]:
# Step 2: Preprocess and Predict Sentiment
def preprocess_articles(articles):
    print("Preprocessing articles...")
    encodings = tokenizer(articles, truncation=True, padding=True, max_length=128, return_tensors='pt')
    return encodings

def predict_sentiment(encodings):
    print("Predicting sentiment...")
    with torch.no_grad():
        outputs = model(**encodings)
        predictions = torch.argmax(outputs.logits, dim=1)
    return predictions

def decode_sentiment(predictions):
    print("Decoding sentiment predictions...")
    sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
    return [sentiment_map[pred.item()] for pred in predictions]

In [26]:
stock_symbol = 'NVDA'  # Example stock symbol
news_titles = search_google_news(f'{stock_symbol} stock')

print(news_titles)
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
state_dict = torch.load('Finetuned_BERT.model.pth', map_location=torch.device(device))
model.load_state_dict(state_dict)
model.eval()

# Preprocess articles
news_encodings = preprocess_articles([article for article in news_titles if article])

# Predict sentiment
predictions = predict_sentiment(news_encodings)

# Decode and calculate average sentiment score
sentiment_labels = decode_sentiment(predictions)
sentiment_scores = {'negative': 0, 'neutral': 1, 'positive': 2}
scores = [sentiment_scores[label] for label in sentiment_labels]
average_sentiment = sum(scores) / len(scores) if scores else 0  # Handle case with no valid scores
print(f'Overall Sentiment Score: {average_sentiment}')

Searching Google News for: NVDA stock
Calling SerpAPI
Going through articles
{
    "search_metadata": {
        "id": "66a6a4d6e4a23dbe722dbdc8",
        "status": "Success",
        "json_endpoint": "https://serpapi.com/searches/9a7cfc16b82c2350/66a6a4d6e4a23dbe722dbdc8.json",
        "created_at": "2024-07-28 20:06:46 UTC",
        "processed_at": "2024-07-28 20:06:46 UTC",
        "google_news_url": "https://news.google.com/search?q=NVDA+stock&gl=US",
        "raw_html_file": "https://serpapi.com/searches/9a7cfc16b82c2350/66a6a4d6e4a23dbe722dbdc8.html",
        "total_time_taken": 2.33
    },
    "search_parameters": {
        "engine": "google_news",
        "gl": "us",
        "q": "NVDA stock"
    },
    "news_results": [
        {
            "position": 1,
            "title": "Nvidia stock market",
            "stories": [
                {
                    "position": 1,
                    "title": "Down 20% From All-Time Highs, is Nvidia Stock a Buy?",
                  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preprocessing articles...
Predicting sentiment...
Decoding sentiment predictions...
Overall Sentiment Score: 0.0
